In [3]:
# Ref: https://machinelearningmastery.com/how-to-use-an-encoder-decoder-lstm-to-echo-sequences-of-random-integers/
# Ref: https://machinelearningmastery.com/use-different-batch-sizes-training-predicting-python-keras/ 
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat

# one hot encode sequence
def one_hot_encode(sequence, n_unique=50): #0-49 of lottery number, though 0 is not used
    encoding = list()
    ##print (sequence.tail(1))
    #for row in sequence.iterrows(): #read each lottery drawresult (this is for dataframe)
    for row in sequence: #read each lottery draw result
        #print ('Row:', row) 
        encoding_row = list()
        for value_str in row:
            #print('Value: ', value_str)
            value = int(value_str) #the csv read its as str, not int
            #if isinstance (value, int): #ignore if not int              
            vector = [0 for _ in range(n_unique)]
            vector[value] = 1
            #print('Vector: ', vector)
            encoding_row.append(vector)
        #print('Encoding Row: ', encoding_row)
        encoding.append(encoding_row)
    return array(encoding)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

# convert encoded sequence to supervised learning
def to_supervised(sequence, n_in, n_out):
    # create lag copies of the sequence
    df = DataFrame(sequence)
    df = concat([df.shift(n_in-i-1) for i in range(n_in)], axis=1)
    # drop rows with missing values
    df.dropna(inplace=True)
    # specify columns for input and output pairs
    values = df.values
    width = sequence.shape[1]
    X = values.reshape(len(values), n_in, width)
    y = values[:, 0:(n_out*width)].reshape(len(values), n_out, width)

In [4]:
# 由2002年7月4日開始至今為止之攪珠結果 (http://mark6.groupsbuy.com/src/download.php)
# All 攪珠結果 http://mark6.groupsbuy.com/src/download.php
import csv
import requests
import pandas as pd 

download = requests.get('http://mark6.groupsbuy.com/data/markSixData.csv')
decoded_content = download.content.decode('utf-8')
marksix_csv = csv.reader(decoded_content.splitlines(), delimiter=',')
dataset = list()
for marksix_result in marksix_csv:
    #print(marksix_result) #each row of the csv file
    marksix_result = marksix_result[2:9] #remove 1st 2col => the year and draw series number and end with the 7 drawed numbers
    dataset.append(marksix_result)
dataset.pop(0) # remove the 1st row, ['Year', 'Draw number', '1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '']
print(dataset[0]) #2-dim

# Below datafame wont work with the encoding
#marksix_results = pd.read_csv('http://mark6.groupsbuy.com/data/markSixData.csv')
#print(marksix_results.head())

# drop all colums and only remain 1st - 7th numbers
#dataset = marksix_results
#dataset.drop('Unnamed: 9', axis=1, inplace=True)
#dataset.drop('Year', axis=1, inplace=True)
#dataset.drop ('Draw number', axis=1, inplace=True)
#print(dataset.tail(1))

['5', '32', '34', '38', '39', '49', '8']


In [6]:
# one hot encode
encoded = one_hot_encode(dataset)

In [25]:
print("length of dataset(total lottery draws): ", len(encoded))
print('Last Lottery result(攪珠結果):')
print('-> 1-hot decoded: ', one_hot_decode(encoded[len(encoded)-1]))
print('-> 1-hot encoded: ',encoded[len(encoded)-1])

length of dataset(total lottery draws):  2381
Last Lottery result(攪珠結果):
-> 1-hot decoded:  [13, 14, 15, 37, 38, 44, 40]
-> 1-hot encoded:  [[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


In [93]:
# IGNORE this for now!!!!!!!!!!
# IGNORE!!!!!!!!!!
#Ref: https://machinelearningmastery.com/use-different-batch-sizes-training-predicting-python-keras/
#Drop this for now as the inputdataset is not a list of sequence, is an array 2-dim

# prepare data for the LSTM
#def get_data(n_in, n_out):
'''
import numpy as np

def get_data(dataset, look_back):
    print(one_hot_decode(dataset[0]), len(dataset))    
    # already one hot encode
    sequence = dataset
    # create X/y pairs
    df = DataFrame(sequence)
    print (df)
    df = concat([df, df.shift(1)], axis=1)
    # drop rows with missing values
    df.dropna(inplace=True)
    # convert to LSTM format
    values = df.values
    width = sequence.shape[1] #7
    X, y = values[:, 0], values[:, 1]
    X = X.reshape(len(X), 1, width)
    return X, y
'''

In [36]:
# prepare data for the LSTM
#def get_data(n_in, n_out):
import numpy as np

def get_data(dataset, look_back):
    dataX, dataY = [], []
    #print(one_hot_decode(dataset[0]), 'Lenght of Dataset: ', len(dataset))
    for i in range (len(dataset)-1):
        # already one hot encode
        X_encoded_sequence = dataset[i]
        Y_encoded_sequence = dataset[i + look_back]
        print('X_encoded_sequence', one_hot_decode(X_encoded_sequence))
        print('Y_encoded_sequence', one_hot_decode(Y_encoded_sequence))
        X_df = DataFrame(X_encoded_sequence)
        Y_df = DataFrame(Y_encoded_sequence)
        # specify columns for input and output pairs
        X_values = X_df.values
        Y_values = Y_df.values
        #print('X length: ', len(X_values), 'Y length: ', len(Y_values)) # 7 , 7
        #print('X_values:', X_values) #already 1-hot encoded
        #X_values = X_values.reshape(7, -1)
        #Y_values = Y_values.reshape(7, -1)
        dataX.append(X_values)
        dataY.append(Y_values)
    return np.array(dataX), np.array(dataY)

In [37]:
# split into train and test sets -> 9.9:0.1
train_size = int(len(encoded) * 0.99)
test_size = len(encoded) - train_size
train, test = encoded[0:train_size,:], encoded[train_size:len(encoded),:]
print("length of dataset(total num of lottery draws): ", len(encoded), "\nlength of train dataset: ", len(train), "\nlength of test dataset: ", len(test))


length of dataset(total num of lottery draws):  2381 
length of train dataset:  2357 
length of test dataset:  24


In [38]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = get_data(train, look_back)
testX, testY = get_data(test, look_back)

X_encoded_sequence [5, 32, 34, 38, 39, 49, 8]
Y_encoded_sequence [10, 20, 21, 33, 40, 44, 34]
X_encoded_sequence [10, 20, 21, 33, 40, 44, 34]
Y_encoded_sequence [10, 20, 26, 28, 35, 48, 13]
X_encoded_sequence [10, 20, 26, 28, 35, 48, 13]
Y_encoded_sequence [14, 19, 23, 36, 44, 46, 7]
X_encoded_sequence [14, 19, 23, 36, 44, 46, 7]
Y_encoded_sequence [9, 24, 25, 27, 36, 38, 41]
X_encoded_sequence [9, 24, 25, 27, 36, 38, 41]
Y_encoded_sequence [7, 13, 14, 20, 29, 32, 28]
X_encoded_sequence [7, 13, 14, 20, 29, 32, 28]
Y_encoded_sequence [11, 22, 24, 28, 29, 38, 20]
X_encoded_sequence [11, 22, 24, 28, 29, 38, 20]
Y_encoded_sequence [1, 11, 29, 33, 36, 44, 37]
X_encoded_sequence [1, 11, 29, 33, 36, 44, 37]
Y_encoded_sequence [9, 12, 25, 31, 43, 44, 5]
X_encoded_sequence [9, 12, 25, 31, 43, 44, 5]
Y_encoded_sequence [1, 5, 6, 31, 39, 46, 44]
X_encoded_sequence [1, 5, 6, 31, 39, 46, 44]
Y_encoded_sequence [1, 23, 25, 32, 43, 44, 30]
X_encoded_sequence [1, 23, 25, 32, 43, 44, 30]
Y_encoded_sequ

X_encoded_sequence [9, 17, 20, 29, 34, 35, 18]
Y_encoded_sequence [8, 29, 31, 32, 36, 38, 12]
X_encoded_sequence [8, 29, 31, 32, 36, 38, 12]
Y_encoded_sequence [6, 28, 39, 40, 43, 45, 42]
X_encoded_sequence [6, 28, 39, 40, 43, 45, 42]
Y_encoded_sequence [8, 14, 17, 23, 28, 43, 26]
X_encoded_sequence [8, 14, 17, 23, 28, 43, 26]
Y_encoded_sequence [1, 27, 28, 29, 31, 49, 6]
X_encoded_sequence [1, 27, 28, 29, 31, 49, 6]
Y_encoded_sequence [13, 16, 19, 31, 39, 47, 29]
X_encoded_sequence [13, 16, 19, 31, 39, 47, 29]
Y_encoded_sequence [2, 12, 21, 37, 47, 49, 11]
X_encoded_sequence [2, 12, 21, 37, 47, 49, 11]
Y_encoded_sequence [9, 27, 30, 33, 44, 48, 37]
X_encoded_sequence [9, 27, 30, 33, 44, 48, 37]
Y_encoded_sequence [10, 19, 26, 30, 32, 44, 29]
X_encoded_sequence [10, 19, 26, 30, 32, 44, 29]
Y_encoded_sequence [3, 14, 27, 31, 38, 44, 26]
X_encoded_sequence [3, 14, 27, 31, 38, 44, 26]
Y_encoded_sequence [3, 11, 18, 22, 24, 37, 49]
X_encoded_sequence [3, 11, 18, 22, 24, 37, 49]
Y_encoded_s

Y_encoded_sequence [7, 22, 24, 25, 40, 41, 1]
X_encoded_sequence [7, 22, 24, 25, 40, 41, 1]
Y_encoded_sequence [2, 3, 4, 8, 16, 18, 6]
X_encoded_sequence [2, 3, 4, 8, 16, 18, 6]
Y_encoded_sequence [1, 9, 14, 39, 45, 48, 3]
X_encoded_sequence [1, 9, 14, 39, 45, 48, 3]
Y_encoded_sequence [8, 20, 33, 36, 46, 49, 18]
X_encoded_sequence [8, 20, 33, 36, 46, 49, 18]
Y_encoded_sequence [1, 19, 20, 34, 42, 43, 15]
X_encoded_sequence [1, 19, 20, 34, 42, 43, 15]
Y_encoded_sequence [9, 10, 26, 31, 35, 41, 42]
X_encoded_sequence [9, 10, 26, 31, 35, 41, 42]
Y_encoded_sequence [10, 21, 26, 30, 31, 42, 48]
X_encoded_sequence [10, 21, 26, 30, 31, 42, 48]
Y_encoded_sequence [14, 23, 33, 35, 36, 43, 37]
X_encoded_sequence [14, 23, 33, 35, 36, 43, 37]
Y_encoded_sequence [7, 10, 18, 23, 37, 39, 17]
X_encoded_sequence [7, 10, 18, 23, 37, 39, 17]
Y_encoded_sequence [2, 3, 4, 25, 36, 47, 8]
X_encoded_sequence [2, 3, 4, 25, 36, 47, 8]
Y_encoded_sequence [6, 21, 28, 36, 44, 49, 32]
X_encoded_sequence [6, 21, 28

Y_encoded_sequence [3, 7, 8, 24, 42, 49, 37]
X_encoded_sequence [3, 7, 8, 24, 42, 49, 37]
Y_encoded_sequence [3, 6, 8, 11, 28, 31, 19]
X_encoded_sequence [3, 6, 8, 11, 28, 31, 19]
Y_encoded_sequence [2, 6, 27, 44, 45, 49, 23]
X_encoded_sequence [2, 6, 27, 44, 45, 49, 23]
Y_encoded_sequence [2, 22, 25, 30, 44, 48, 46]
X_encoded_sequence [2, 22, 25, 30, 44, 48, 46]
Y_encoded_sequence [1, 9, 21, 35, 36, 47, 33]
X_encoded_sequence [1, 9, 21, 35, 36, 47, 33]
Y_encoded_sequence [2, 9, 10, 13, 15, 24, 19]
X_encoded_sequence [2, 9, 10, 13, 15, 24, 19]
Y_encoded_sequence [2, 10, 13, 22, 24, 25, 5]
X_encoded_sequence [2, 10, 13, 22, 24, 25, 5]
Y_encoded_sequence [2, 5, 10, 14, 29, 37, 3]
X_encoded_sequence [2, 5, 10, 14, 29, 37, 3]
Y_encoded_sequence [6, 9, 13, 21, 46, 47, 12]
X_encoded_sequence [6, 9, 13, 21, 46, 47, 12]
Y_encoded_sequence [2, 12, 14, 38, 39, 43, 48]
X_encoded_sequence [2, 12, 14, 38, 39, 43, 48]
Y_encoded_sequence [11, 12, 17, 22, 31, 43, 45]
X_encoded_sequence [11, 12, 17, 22

X_encoded_sequence [2, 18, 30, 34, 37, 41, 28]
Y_encoded_sequence [1, 3, 10, 31, 34, 35, 27]
X_encoded_sequence [1, 3, 10, 31, 34, 35, 27]
Y_encoded_sequence [3, 11, 13, 20, 38, 49, 19]
X_encoded_sequence [3, 11, 13, 20, 38, 49, 19]
Y_encoded_sequence [3, 10, 13, 35, 48, 49, 33]
X_encoded_sequence [3, 10, 13, 35, 48, 49, 33]
Y_encoded_sequence [7, 8, 12, 37, 41, 48, 40]
X_encoded_sequence [7, 8, 12, 37, 41, 48, 40]
Y_encoded_sequence [1, 13, 21, 22, 24, 40, 27]
X_encoded_sequence [1, 13, 21, 22, 24, 40, 27]
Y_encoded_sequence [23, 24, 26, 30, 47, 49, 43]
X_encoded_sequence [23, 24, 26, 30, 47, 49, 43]
Y_encoded_sequence [4, 9, 10, 14, 15, 49, 5]
X_encoded_sequence [4, 9, 10, 14, 15, 49, 5]
Y_encoded_sequence [2, 3, 15, 30, 32, 46, 23]
X_encoded_sequence [2, 3, 15, 30, 32, 46, 23]
Y_encoded_sequence [12, 23, 25, 31, 44, 45, 3]
X_encoded_sequence [12, 23, 25, 31, 44, 45, 3]
Y_encoded_sequence [1, 4, 5, 8, 13, 43, 22]
X_encoded_sequence [1, 4, 5, 8, 13, 43, 22]
Y_encoded_sequence [3, 16, 

...

Y_encoded_sequence [8, 15, 31, 35, 42, 44, 39]
X_encoded_sequence [8, 15, 31, 35, 42, 44, 39]
Y_encoded_sequence [3, 14, 16, 18, 38, 49, 29]
X_encoded_sequence [3, 14, 16, 18, 38, 49, 29]
Y_encoded_sequence [8, 18, 19, 28, 37, 44, 13]
X_encoded_sequence [8, 18, 19, 28, 37, 44, 13]
Y_encoded_sequence [10, 21, 24, 33, 40, 47, 5]
X_encoded_sequence [10, 21, 24, 33, 40, 47, 5]
Y_encoded_sequence [16, 21, 27, 37, 40, 43, 14]
X_encoded_sequence [16, 21, 27, 37, 40, 43, 14]
Y_encoded_sequence [1, 2, 14, 37, 43, 49, 24]
X_encoded_sequence [1, 2, 14, 37, 43, 49, 24]
Y_encoded_sequence [14, 15, 16, 25, 27, 30, 11]
X_encoded_sequence [14, 15, 16, 25, 27, 30, 11]
Y_encoded_sequence [5, 7, 9, 13, 15, 29, 47]
X_encoded_sequence [5, 7, 9, 13, 15, 29, 47]
Y_encoded_sequence [8, 16, 21, 30, 42, 46, 32]
X_encoded_sequence [8, 16, 21, 30, 42, 46, 32]
Y_encoded_sequence [17, 18, 20, 26, 31, 32, 6]
X_encoded_sequence [17, 18, 20, 26, 31, 32, 6]
Y_encoded_sequence [4, 5, 10, 13, 28, 35, 12]
X_encoded_sequen

Y_encoded_sequence [4, 10, 28, 37, 42, 48, 8]
X_encoded_sequence [4, 10, 28, 37, 42, 48, 8]
Y_encoded_sequence [7, 12, 13, 25, 34, 38, 15]
X_encoded_sequence [7, 12, 13, 25, 34, 38, 15]
Y_encoded_sequence [5, 10, 11, 25, 27, 44, 7]
X_encoded_sequence [5, 10, 11, 25, 27, 44, 7]
Y_encoded_sequence [6, 16, 27, 34, 44, 48, 1]
X_encoded_sequence [6, 16, 27, 34, 44, 48, 1]
Y_encoded_sequence [4, 14, 28, 42, 43, 47, 3]
X_encoded_sequence [4, 14, 28, 42, 43, 47, 3]
Y_encoded_sequence [10, 20, 24, 28, 33, 45, 19]
X_encoded_sequence [10, 20, 24, 28, 33, 45, 19]
Y_encoded_sequence [13, 15, 24, 28, 41, 45, 20]
X_encoded_sequence [13, 15, 24, 28, 41, 45, 20]
Y_encoded_sequence [12, 13, 31, 33, 35, 47, 21]
X_encoded_sequence [12, 13, 31, 33, 35, 47, 21]
Y_encoded_sequence [13, 15, 25, 40, 41, 48, 19]
X_encoded_sequence [13, 15, 25, 40, 41, 48, 19]
Y_encoded_sequence [9, 20, 30, 33, 38, 39, 37]
X_encoded_sequence [9, 20, 30, 33, 38, 39, 37]
Y_encoded_sequence [17, 21, 22, 36, 41, 44, 38]
X_encoded_se

Y_encoded_sequence [14, 20, 21, 31, 34, 42, 25]
X_encoded_sequence [14, 20, 21, 31, 34, 42, 25]
Y_encoded_sequence [8, 11, 13, 30, 34, 49, 2]
X_encoded_sequence [8, 11, 13, 30, 34, 49, 2]
Y_encoded_sequence [8, 25, 26, 28, 38, 43, 22]
X_encoded_sequence [8, 25, 26, 28, 38, 43, 22]
Y_encoded_sequence [11, 15, 33, 41, 43, 48, 27]
X_encoded_sequence [11, 15, 33, 41, 43, 48, 27]
Y_encoded_sequence [3, 4, 5, 26, 45, 48, 14]
X_encoded_sequence [3, 4, 5, 26, 45, 48, 14]
Y_encoded_sequence [7, 16, 32, 37, 38, 46, 22]
X_encoded_sequence [7, 16, 32, 37, 38, 46, 22]
Y_encoded_sequence [1, 13, 19, 21, 38, 49, 23]
X_encoded_sequence [1, 13, 19, 21, 38, 49, 23]
Y_encoded_sequence [24, 25, 34, 35, 44, 49, 30]
X_encoded_sequence [24, 25, 34, 35, 44, 49, 30]
Y_encoded_sequence [3, 18, 21, 25, 31, 42, 26]
X_encoded_sequence [3, 18, 21, 25, 31, 42, 26]
Y_encoded_sequence [4, 16, 19, 27, 41, 44, 25]
X_encoded_sequence [4, 16, 19, 27, 41, 44, 25]
Y_encoded_sequence [5, 30, 38, 43, 44, 47, 14]
X_encoded_seq

X_encoded_sequence [9, 16, 22, 38, 40, 46, 11]
Y_encoded_sequence [1, 7, 12, 18, 27, 33, 20]
X_encoded_sequence [1, 7, 12, 18, 27, 33, 20]
Y_encoded_sequence [1, 5, 7, 9, 10, 15, 3]
X_encoded_sequence [1, 5, 7, 9, 10, 15, 3]
Y_encoded_sequence [7, 11, 20, 28, 41, 49, 32]
X_encoded_sequence [7, 11, 20, 28, 41, 49, 32]
Y_encoded_sequence [2, 3, 17, 34, 38, 48, 14]
X_encoded_sequence [2, 3, 17, 34, 38, 48, 14]
Y_encoded_sequence [7, 13, 17, 23, 29, 46, 49]
X_encoded_sequence [7, 13, 17, 23, 29, 46, 49]
Y_encoded_sequence [23, 26, 27, 39, 40, 48, 7]
X_encoded_sequence [23, 26, 27, 39, 40, 48, 7]
Y_encoded_sequence [5, 7, 18, 22, 33, 43, 27]
X_encoded_sequence [5, 7, 18, 22, 33, 43, 27]
Y_encoded_sequence [14, 18, 27, 32, 37, 38, 1]
X_encoded_sequence [14, 18, 27, 32, 37, 38, 1]
Y_encoded_sequence [4, 18, 23, 28, 36, 45, 33]
X_encoded_sequence [4, 18, 23, 28, 36, 45, 33]
Y_encoded_sequence [11, 15, 24, 25, 33, 40, 42]
X_encoded_sequence [11, 15, 24, 25, 33, 40, 42]
Y_encoded_sequence [8, 24

X_encoded_sequence [8, 12, 20, 33, 35, 45, 42]
Y_encoded_sequence [2, 15, 17, 18, 21, 44, 14]
X_encoded_sequence [2, 15, 17, 18, 21, 44, 14]
Y_encoded_sequence [1, 5, 10, 20, 21, 25, 43]
X_encoded_sequence [1, 5, 10, 20, 21, 25, 43]
Y_encoded_sequence [12, 24, 30, 31, 32, 34, 41]
X_encoded_sequence [12, 24, 30, 31, 32, 34, 41]
Y_encoded_sequence [3, 24, 40, 44, 45, 49, 8]
X_encoded_sequence [3, 24, 40, 44, 45, 49, 8]
Y_encoded_sequence [3, 14, 18, 28, 31, 40, 38]
X_encoded_sequence [3, 14, 18, 28, 31, 40, 38]
Y_encoded_sequence [12, 19, 25, 31, 36, 49, 11]
X_encoded_sequence [12, 19, 25, 31, 36, 49, 11]
Y_encoded_sequence [1, 6, 9, 19, 21, 23, 43]
X_encoded_sequence [1, 6, 9, 19, 21, 23, 43]
Y_encoded_sequence [5, 8, 9, 19, 32, 44, 39]
X_encoded_sequence [5, 8, 9, 19, 32, 44, 39]
Y_encoded_sequence [3, 16, 23, 30, 43, 46, 26]
X_encoded_sequence [3, 16, 23, 30, 43, 46, 26]
Y_encoded_sequence [14, 23, 29, 36, 37, 47, 25]
X_encoded_sequence [14, 23, 29, 36, 37, 47, 25]
Y_encoded_sequence 

In [41]:
# The LSTM network expects the input data (X) to be provided with a specific array structure in the form of: [samples, time steps, features].
# Currently, our data is in the form: [samples, features]
print("Num of Training Sample(trainX): ", trainX.shape[0], "Num of features (lottery result numbers): ", trainX.shape[1], trainX.shape[2]) #7(7 num), 50(0-49 encoded)
print("Num of Testing Sample(testX): ", testX.shape[0], "Num of features (lottery result numbers): ", testX.shape[1], testX.shape[2])

Num of Training Sample(trainX):  2356 Num of features (lottery result numbers):  7 50
Num of Testing Sample(testX):  23 Num of features (lottery result numbers):  7 50


In [43]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
import tensorflow as tf

#GPU
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

np.random.seed(7)

# define LSTM
n_in = 7
n_out = 1
encoded_length = 50

# https://www.mathsisfun.com/greatest-common-factor-tool.html 
# http://www.mathwarehouse.com/arithmetic/factorization-calculator.php
#batch_size = 82 #82x29=2378 all data
#batch_size = 128 #104x16=1664 training data (2, 4, 8, 13, 16, 26, 32, 52, 64, 104, 128, 208, 416, 832)
#batch_size = 23 # (1, 23, 31, 713) testing data

batch_size =124 #(1, 2, 4, 19, 31, 38, 62, 76, 124, 589, 1178, 2356)
#batch_size = 1 #use for 1 step prediction, for n step predict->model.fit's validation data will complain different batch size use for trainning
model = Sequential()
model.add(LSTM(150, batch_input_shape=(batch_size, n_in, encoded_length), return_sequences=True, stateful=True))
#model.add(LSTM(150, batch_input_shape=(batch_size, trainX.shape[1], trainX.shape[2]), return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(encoded_length, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

Using CNTK backend


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4752115583713134696
]


In [44]:
# train LSTM
for epoch in range(300):
    # generate new random sequence
    #X,y = get_data(encoded, 1)
    #X, y = get_data(train, look_back)
    # fit model for one epoch on this sequence
    #model.fit(X, y, epochs=5, batch_size=batch_size, verbose=2, shuffle=False)
    #model.fit(trainX, trainY, epochs=1, batch_size=batch_size, verbose=2, shuffle=False, validation_data=(testX, testY))
    print('n_epoch : ', epoch)
    model.fit(trainX, trainY, epochs=15, batch_size=batch_size, verbose=2, shuffle=False)
    model.reset_states()

n_epoch :  0
Epoch 1/15
 - 4s - loss: 3.9089 - acc: 0.0208
Epoch 2/15
 - 5s - loss: 3.9182 - acc: 0.0223
Epoch 3/15
 - 5s - loss: 3.8966 - acc: 0.0233
Epoch 4/15
 - 5s - loss: 3.8980 - acc: 0.0231
Epoch 5/15
 - 5s - loss: 3.8971 - acc: 0.0270
Epoch 6/15
 - 5s - loss: 3.8913 - acc: 0.0280
Epoch 7/15
 - 5s - loss: 3.8838 - acc: 0.0291
Epoch 8/15
 - 4s - loss: 3.8787 - acc: 0.0297
Epoch 9/15
 - 4s - loss: 3.8677 - acc: 0.0327
Epoch 10/15
 - 4s - loss: 3.8385 - acc: 0.0355
Epoch 11/15
 - 3s - loss: 3.6851 - acc: 0.0441
Epoch 12/15
 - 3s - loss: 3.5114 - acc: 0.0541
Epoch 13/15
 - 3s - loss: 3.4907 - acc: 0.0583
Epoch 14/15
 - 4s - loss: 3.5033 - acc: 0.0571
Epoch 15/15
 - 3s - loss: 3.4744 - acc: 0.0639
n_epoch :  1
Epoch 1/15
 - 3s - loss: 3.5129 - acc: 0.0604
Epoch 2/15
 - 4s - loss: 3.4612 - acc: 0.0708
Epoch 3/15
 - 3s - loss: 3.4293 - acc: 0.0700
Epoch 4/15
 - 3s - loss: 3.4396 - acc: 0.0729
Epoch 5/15
 - 3s - loss: 3.4422 - acc: 0.0723
Epoch 6/15
 - 3s - loss: 3.4086 - acc: 0.0788
Ep

 - 2s - loss: 2.6124 - acc: 0.2593
Epoch 10/15
 - 2s - loss: 2.6063 - acc: 0.2629
Epoch 11/15
 - 2s - loss: 2.6123 - acc: 0.2619
Epoch 12/15
 - 2s - loss: 2.5897 - acc: 0.2701
Epoch 13/15
 - 3s - loss: 2.6019 - acc: 0.2647
Epoch 14/15
 - 2s - loss: 2.5974 - acc: 0.2653
Epoch 15/15
 - 2s - loss: 2.5751 - acc: 0.2749
n_epoch :  12
Epoch 1/15
 - 2s - loss: 2.6307 - acc: 0.2603
Epoch 2/15
 - 3s - loss: 2.5522 - acc: 0.2788
Epoch 3/15
 - 3s - loss: 2.5605 - acc: 0.2761
Epoch 4/15
 - 2s - loss: 2.5633 - acc: 0.2735
Epoch 5/15
 - 2s - loss: 2.5537 - acc: 0.2780
Epoch 6/15
 - 2s - loss: 2.5305 - acc: 0.2848
Epoch 7/15
 - 2s - loss: 2.5352 - acc: 0.2809
Epoch 8/15
 - 2s - loss: 2.5211 - acc: 0.2885
Epoch 9/15
 - 2s - loss: 2.5223 - acc: 0.2834
Epoch 10/15
 - 2s - loss: 2.5225 - acc: 0.2867
Epoch 11/15
 - 2s - loss: 2.4966 - acc: 0.2932
Epoch 12/15
 - 3s - loss: 2.4797 - acc: 0.2974
Epoch 13/15
 - 2s - loss: 2.4800 - acc: 0.2945
Epoch 14/15
 - 2s - loss: 2.4903 - acc: 0.2930
Epoch 15/15
 - 2s - 

 - 3s - loss: 1.5145 - acc: 0.5738
Epoch 3/15
 - 2s - loss: 1.5034 - acc: 0.5776
Epoch 4/15
 - 2s - loss: 1.4833 - acc: 0.5894
Epoch 5/15
 - 2s - loss: 1.4776 - acc: 0.5919
Epoch 6/15
 - 3s - loss: 1.4783 - acc: 0.5847
Epoch 7/15
 - 2s - loss: 1.4482 - acc: 0.5961
Epoch 8/15
 - 2s - loss: 1.4562 - acc: 0.5964
Epoch 9/15
 - 2s - loss: 1.4517 - acc: 0.5905
Epoch 10/15
 - 2s - loss: 1.4494 - acc: 0.5958
Epoch 11/15
 - 3s - loss: 1.4429 - acc: 0.5990
Epoch 12/15
 - 2s - loss: 1.4560 - acc: 0.5901
Epoch 13/15
 - 2s - loss: 1.4560 - acc: 0.5934
Epoch 14/15
 - 2s - loss: 1.4728 - acc: 0.5851
Epoch 15/15
 - 2s - loss: 1.4575 - acc: 0.5882
n_epoch :  24
Epoch 1/15
 - 3s - loss: 1.6206 - acc: 0.5392
Epoch 2/15
 - 2s - loss: 1.4810 - acc: 0.5863
Epoch 3/15
 - 2s - loss: 1.4656 - acc: 0.5901
Epoch 4/15
 - 2s - loss: 1.4220 - acc: 0.6027
Epoch 5/15
 - 3s - loss: 1.4490 - acc: 0.5949
Epoch 6/15
 - 2s - loss: 1.4358 - acc: 0.5943
Epoch 7/15
 - 2s - loss: 1.4500 - acc: 0.5894
Epoch 8/15
 - 2s - loss: 

Epoch 11/15
 - 2s - loss: 0.8889 - acc: 0.7559
Epoch 12/15
 - 2s - loss: 0.9198 - acc: 0.7416
Epoch 13/15
 - 2s - loss: 0.9116 - acc: 0.7448
Epoch 14/15
 - 2s - loss: 0.8964 - acc: 0.7507
Epoch 15/15
 - 2s - loss: 0.8936 - acc: 0.7535
n_epoch :  35
Epoch 1/15
 - 2s - loss: 1.0850 - acc: 0.6952
Epoch 2/15
 - 3s - loss: 0.8926 - acc: 0.7528
Epoch 3/15
 - 2s - loss: 0.8809 - acc: 0.7559
Epoch 4/15
 - 2s - loss: 0.9001 - acc: 0.7467
Epoch 5/15
 - 2s - loss: 0.8828 - acc: 0.7584
Epoch 6/15
 - 2s - loss: 0.8704 - acc: 0.7610
Epoch 7/15
 - 2s - loss: 0.8671 - acc: 0.7616
Epoch 8/15
 - 2s - loss: 0.8928 - acc: 0.7522
Epoch 9/15
 - 2s - loss: 0.9218 - acc: 0.7423
Epoch 10/15
 - 2s - loss: 0.9158 - acc: 0.7419
Epoch 11/15
 - 3s - loss: 0.9261 - acc: 0.7407
Epoch 12/15
 - 2s - loss: 0.8987 - acc: 0.7489
Epoch 13/15
 - 3s - loss: 0.9184 - acc: 0.7382
Epoch 14/15
 - 2s - loss: 0.9353 - acc: 0.7347
Epoch 15/15
 - 2s - loss: 0.9476 - acc: 0.7330
n_epoch :  36
Epoch 1/15
 - 2s - loss: 1.1302 - acc: 0.

Epoch 4/15
 - 2s - loss: 0.5614 - acc: 0.8521
Epoch 5/15
 - 2s - loss: 0.5685 - acc: 0.8496
Epoch 6/15
 - 2s - loss: 0.5766 - acc: 0.8426
Epoch 7/15
 - 3s - loss: 0.5728 - acc: 0.8502
Epoch 8/15
 - 2s - loss: 0.5587 - acc: 0.8497
Epoch 9/15
 - 2s - loss: 0.5436 - acc: 0.8557
Epoch 10/15
 - 2s - loss: 0.5597 - acc: 0.8527
Epoch 11/15
 - 3s - loss: 0.5524 - acc: 0.8528
Epoch 12/15
 - 3s - loss: 0.5657 - acc: 0.8463
Epoch 13/15
 - 2s - loss: 0.5566 - acc: 0.8541
Epoch 14/15
 - 2s - loss: 0.5564 - acc: 0.8510
Epoch 15/15
 - 2s - loss: 0.5453 - acc: 0.8575
n_epoch :  47
Epoch 1/15
 - 3s - loss: 0.7724 - acc: 0.7834
Epoch 2/15
 - 2s - loss: 0.5547 - acc: 0.8542
Epoch 3/15
 - 2s - loss: 0.5528 - acc: 0.8563
Epoch 4/15
 - 2s - loss: 0.5687 - acc: 0.8475
Epoch 5/15
 - 3s - loss: 0.5534 - acc: 0.8499
Epoch 6/15
 - 2s - loss: 0.5597 - acc: 0.8471
Epoch 7/15
 - 2s - loss: 0.5678 - acc: 0.8462
Epoch 8/15
 - 2s - loss: 0.5501 - acc: 0.8492
Epoch 9/15
 - 2s - loss: 0.5602 - acc: 0.8494
Epoch 10/15
 -

...

Epoch 7/15
 - 2s - loss: 0.0028 - acc: 1.0000
Epoch 8/15
 - 2s - loss: 0.0027 - acc: 1.0000
Epoch 9/15
 - 2s - loss: 0.0027 - acc: 1.0000
Epoch 10/15
 - 2s - loss: 0.0026 - acc: 1.0000
Epoch 11/15
 - 2s - loss: 0.0026 - acc: 1.0000
Epoch 12/15
 - 2s - loss: 0.0026 - acc: 1.0000
Epoch 13/15
 - 2s - loss: 0.0025 - acc: 1.0000
Epoch 14/15
 - 2s - loss: 0.0025 - acc: 1.0000
Epoch 15/15
 - 2s - loss: 0.0025 - acc: 1.0000
n_epoch :  243
Epoch 1/15
 - 2s - loss: 0.0284 - acc: 0.9929
Epoch 2/15
 - 2s - loss: 0.0035 - acc: 1.0000
Epoch 3/15
 - 2s - loss: 0.0032 - acc: 1.0000
Epoch 4/15
 - 2s - loss: 0.0029 - acc: 1.0000
Epoch 5/15
 - 2s - loss: 0.0027 - acc: 1.0000
Epoch 6/15
 - 2s - loss: 0.0026 - acc: 1.0000
Epoch 7/15
 - 2s - loss: 0.0025 - acc: 1.0000
Epoch 8/15
 - 2s - loss: 0.0025 - acc: 1.0000
Epoch 9/15
 - 2s - loss: 0.0025 - acc: 1.0000
Epoch 10/15
 - 2s - loss: 0.0025 - acc: 1.0000
Epoch 11/15
 - 2s - loss: 0.0024 - acc: 1.0000
Epoch 12/15
 - 2s - loss: 0.0024 - acc: 1.0000
Epoch 13/1

Epoch 15/15
 - 3s - loss: 0.0053 - acc: 1.0000
n_epoch :  254
Epoch 1/15
 - 5s - loss: 0.0394 - acc: 0.9901
Epoch 2/15
 - 5s - loss: 0.0078 - acc: 0.9997
Epoch 3/15
 - 5s - loss: 0.0061 - acc: 1.0000
Epoch 4/15
 - 5s - loss: 0.0053 - acc: 1.0000
Epoch 5/15
 - 4s - loss: 0.0049 - acc: 1.0000
Epoch 6/15
 - 4s - loss: 0.0047 - acc: 1.0000
Epoch 7/15
 - 4s - loss: 0.0046 - acc: 1.0000
Epoch 8/15
 - 4s - loss: 0.0044 - acc: 1.0000
Epoch 9/15
 - 4s - loss: 0.0043 - acc: 1.0000
Epoch 10/15
 - 3s - loss: 0.0042 - acc: 1.0000
Epoch 11/15
 - 3s - loss: 0.0042 - acc: 1.0000
Epoch 12/15
 - 3s - loss: 0.0041 - acc: 1.0000
Epoch 13/15
 - 3s - loss: 0.0040 - acc: 1.0000
Epoch 14/15
 - 4s - loss: 0.0040 - acc: 1.0000
Epoch 15/15
 - 4s - loss: 0.0039 - acc: 1.0000
n_epoch :  255
Epoch 1/15
 - 4s - loss: 0.0325 - acc: 0.9925
Epoch 2/15
 - 4s - loss: 0.0050 - acc: 0.9999
Epoch 3/15
 - 5s - loss: 0.0043 - acc: 1.0000
Epoch 4/15
 - 4s - loss: 0.0041 - acc: 1.0000
Epoch 5/15
 - 5s - loss: 0.0039 - acc: 1.00

Epoch 8/15
 - 4s - loss: 0.0022 - acc: 1.0000
Epoch 9/15
 - 4s - loss: 0.0022 - acc: 1.0000
Epoch 10/15
 - 4s - loss: 0.0022 - acc: 1.0000
Epoch 11/15
 - 3s - loss: 0.0022 - acc: 1.0000
Epoch 12/15
 - 3s - loss: 0.0022 - acc: 1.0000
Epoch 13/15
 - 6s - loss: 0.0021 - acc: 1.0000
Epoch 14/15
 - 3s - loss: 0.0021 - acc: 1.0000
Epoch 15/15
 - 4s - loss: 0.0021 - acc: 1.0000
n_epoch :  266
Epoch 1/15
 - 4s - loss: 0.0251 - acc: 0.9932
Epoch 2/15
 - 3s - loss: 0.0026 - acc: 1.0000
Epoch 3/15
 - 3s - loss: 0.0023 - acc: 1.0000
Epoch 4/15
 - 4s - loss: 0.0022 - acc: 1.0000
Epoch 5/15
 - 3s - loss: 0.0021 - acc: 1.0000
Epoch 6/15
 - 3s - loss: 0.0021 - acc: 1.0000
Epoch 7/15
 - 5s - loss: 0.0020 - acc: 1.0000
Epoch 8/15
 - 4s - loss: 0.0020 - acc: 1.0000
Epoch 9/15
 - 3s - loss: 0.0020 - acc: 1.0000
Epoch 10/15
 - 4s - loss: 0.0020 - acc: 1.0000
Epoch 11/15
 - 4s - loss: 0.0020 - acc: 1.0000
Epoch 12/15
 - 5s - loss: 0.0020 - acc: 1.0000
Epoch 13/15
 - 4s - loss: 0.0019 - acc: 1.0000
Epoch 14/

n_epoch :  277
Epoch 1/15
 - 3s - loss: 0.0881 - acc: 0.9734
Epoch 2/15
 - 3s - loss: 0.0362 - acc: 0.9913
Epoch 3/15
 - 3s - loss: 0.0232 - acc: 0.9962
Epoch 4/15
 - 4s - loss: 0.0168 - acc: 0.9982
Epoch 5/15
 - 3s - loss: 0.0127 - acc: 0.9988
Epoch 6/15
 - 3s - loss: 0.0100 - acc: 0.9998
Epoch 7/15
 - 3s - loss: 0.0081 - acc: 0.9999
Epoch 8/15
 - 3s - loss: 0.0070 - acc: 1.0000
Epoch 9/15
 - 3s - loss: 0.0064 - acc: 1.0000
Epoch 10/15
 - 3s - loss: 0.0060 - acc: 1.0000
Epoch 11/15
 - 3s - loss: 0.0057 - acc: 1.0000
Epoch 12/15
 - 3s - loss: 0.0054 - acc: 1.0000
Epoch 13/15
 - 3s - loss: 0.0052 - acc: 1.0000
Epoch 14/15
 - 3s - loss: 0.0050 - acc: 1.0000
Epoch 15/15
 - 3s - loss: 0.0048 - acc: 1.0000
n_epoch :  278
Epoch 1/15
 - 3s - loss: 0.0417 - acc: 0.9895
Epoch 2/15
 - 3s - loss: 0.0112 - acc: 0.9988
Epoch 3/15
 - 3s - loss: 0.0081 - acc: 0.9998
Epoch 4/15
 - 3s - loss: 0.0061 - acc: 0.9999
Epoch 5/15
 - 3s - loss: 0.0053 - acc: 1.0000
Epoch 6/15
 - 3s - loss: 0.0048 - acc: 1.000

Epoch 9/15
 - 3s - loss: 0.0029 - acc: 1.0000
Epoch 10/15
 - 3s - loss: 0.0028 - acc: 1.0000
Epoch 11/15
 - 3s - loss: 0.0028 - acc: 1.0000
Epoch 12/15
 - 4s - loss: 0.0027 - acc: 1.0000
Epoch 13/15
 - 3s - loss: 0.0027 - acc: 1.0000
Epoch 14/15
 - 3s - loss: 0.0027 - acc: 1.0000
Epoch 15/15
 - 3s - loss: 0.0026 - acc: 1.0000
n_epoch :  289
Epoch 1/15
 - 3s - loss: 0.0264 - acc: 0.9930
Epoch 2/15
 - 3s - loss: 0.0031 - acc: 1.0000
Epoch 3/15
 - 3s - loss: 0.0028 - acc: 1.0000
Epoch 4/15
 - 3s - loss: 0.0027 - acc: 1.0000
Epoch 5/15
 - 3s - loss: 0.0026 - acc: 1.0000
Epoch 6/15
 - 3s - loss: 0.0025 - acc: 1.0000
Epoch 7/15
 - 3s - loss: 0.0025 - acc: 1.0000
Epoch 8/15
 - 3s - loss: 0.0025 - acc: 1.0000
Epoch 9/15
 - 3s - loss: 0.0024 - acc: 1.0000
Epoch 10/15
 - 3s - loss: 0.0024 - acc: 1.0000
Epoch 11/15
 - 4s - loss: 0.0024 - acc: 1.0000
Epoch 12/15
 - 4s - loss: 0.0023 - acc: 1.0000
Epoch 13/15
 - 4s - loss: 0.0023 - acc: 1.0000
Epoch 14/15
 - 4s - loss: 0.0023 - acc: 1.0000
Epoch 15

In [45]:
# re-define the batch size
n_batch = 1
new_encoded_length = 50
# re-define model
new_model = Sequential()
new_model.add(LSTM(150, batch_input_shape=(n_batch, n_in, new_encoded_length), return_sequences=True, stateful=True))
new_model.add(TimeDistributed(Dense(encoded_length, activation='softmax')))

# copy weights
old_weights = model.get_weights()
new_model.set_weights(old_weights)
# compile model
new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])


In [67]:
# online forecast
for i in range(len(testX)):
    test_X, test_Y = testX[i], testY[i]
    #print('X: ', one_hot_decode(test_X), 'Y: ', one_hot_decode(test_Y))
    test_X = test_X.reshape(-1, 7, 50)
    #yhat = new_model.predict(test_X, batch_size=n_batch)
    print('testX: ', one_hot_decode(test_X[0]))
    yhat = new_model.predict(test_X)
    print('>Expected= ',one_hot_decode(test_Y), ' Predict= ', one_hot_decode(yhat[0]))
    

testX:  [3, 6, 11, 22, 45, 46, 1]
>Expected=  [3, 5, 28, 33, 40, 49, 32]  Predict=  [9, 16, 18, 21, 46, 48, 35]
testX:  [3, 5, 28, 33, 40, 49, 32]
>Expected=  [4, 15, 30, 33, 38, 41, 16]  Predict=  [7, 13, 20, 37, 45, 47, 14]
testX:  [4, 15, 30, 33, 38, 41, 16]
>Expected=  [6, 10, 24, 34, 40, 45, 9]  Predict=  [3, 4, 16, 21, 46, 40, 34]
testX:  [6, 10, 24, 34, 40, 45, 9]
>Expected=  [5, 32, 34, 36, 38, 43, 25]  Predict=  [22, 2, 27, 42, 49, 44, 8]
testX:  [5, 32, 34, 36, 38, 43, 25]
>Expected=  [11, 23, 35, 41, 46, 49, 47]  Predict=  [18, 29, 30, 37, 41, 47, 2]
testX:  [11, 23, 35, 41, 46, 49, 47]
>Expected=  [30, 31, 34, 40, 41, 42, 33]  Predict=  [13, 17, 18, 35, 39, 35, 2]
testX:  [30, 31, 34, 40, 41, 42, 33]
>Expected=  [2, 10, 24, 28, 35, 39, 27]  Predict=  [13, 2, 39, 2, 18, 40, 22]
testX:  [2, 10, 24, 28, 35, 39, 27]
>Expected=  [1, 7, 20, 21, 35, 48, 22]  Predict=  [14, 13, 26, 42, 41, 49, 20]
testX:  [1, 7, 20, 21, 35, 48, 22]
>Expected=  [2, 7, 21, 31, 33, 42, 24]  Predict=  

In [54]:
this_sequence = one_hot_encode(np.array([['13', '14', '15', '37', '38', '44', '40'], ['13', '14', '15', '37', '38', '44', '40']]))          
this_X = DataFrame(this_sequence[0])
X_value = this_X.values
print('X_value: ', one_hot_decode(X_value))
X_value = X_value.reshape(1, 7, 50)
predict_Y = new_model.predict(X_value)
print(' Predict= ', one_hot_decode(predict_Y[0]))

X_value:  [13, 14, 15, 37, 38, 44, 40]
 Predict=  [1, 25, 14, 25, 30, 41, 47]


In [56]:
# evaluate LSTM with original training data
predict = model.predict(trainX, batch_size, verbose=1)
# decode all pairs
for i in range(len(trainX)):
    print('Expected:', one_hot_decode(trainY[i]), 'Predicted', one_hot_decode(predict[i]))

2356/2356 [==============================] - 2s 974us/step
Expected: [10, 20, 21, 33, 40, 44, 34] Predicted [10, 20, 21, 33, 40, 44, 34]
Expected: [10, 20, 26, 28, 35, 48, 13] Predicted [10, 20, 26, 28, 35, 48, 13]
Expected: [14, 19, 23, 36, 44, 46, 7] Predicted [14, 19, 23, 36, 44, 46, 7]
Expected: [9, 24, 25, 27, 36, 38, 41] Predicted [9, 24, 25, 27, 36, 38, 41]
Expected: [7, 13, 14, 20, 29, 32, 28] Predicted [7, 13, 14, 20, 29, 32, 28]
Expected: [11, 22, 24, 28, 29, 38, 20] Predicted [11, 22, 24, 28, 29, 38, 20]
Expected: [1, 11, 29, 33, 36, 44, 37] Predicted [1, 11, 29, 33, 36, 44, 37]
Expected: [9, 12, 25, 31, 43, 44, 5] Predicted [9, 12, 25, 31, 43, 44, 5]
Expected: [1, 5, 6, 31, 39, 46, 44] Predicted [1, 5, 6, 31, 39, 46, 44]
Expected: [1, 23, 25, 32, 43, 44, 30] Predicted [1, 23, 25, 32, 43, 44, 30]
Expected: [6, 22, 25, 26, 27, 45, 36] Predicted [6, 22, 25, 26, 27, 45, 36]
Expected: [3, 7, 11, 22, 28, 36, 18] Predicted [3, 7, 11, 22, 28, 36, 18]
Expected: [9, 10, 11, 15, 19, 2

Expected: [22, 28, 31, 36, 39, 49, 32] Predicted [22, 28, 31, 36, 39, 49, 32]
Expected: [12, 13, 14, 20, 23, 28, 22] Predicted [12, 13, 14, 20, 23, 28, 22]
Expected: [3, 13, 15, 39, 43, 44, 31] Predicted [3, 13, 15, 39, 43, 44, 31]
Expected: [4, 12, 14, 22, 46, 48, 23] Predicted [4, 12, 14, 22, 46, 48, 23]
Expected: [18, 19, 33, 43, 45, 48, 11] Predicted [18, 19, 33, 43, 45, 48, 11]
Expected: [4, 16, 24, 26, 32, 39, 6] Predicted [4, 16, 24, 26, 32, 39, 6]
Expected: [4, 6, 20, 35, 37, 39, 41] Predicted [4, 6, 20, 35, 37, 39, 41]
Expected: [7, 17, 27, 29, 30, 34, 36] Predicted [7, 17, 27, 29, 30, 34, 36]
Expected: [7, 12, 20, 33, 43, 44, 38] Predicted [7, 12, 20, 33, 43, 44, 38]
Expected: [5, 17, 21, 27, 31, 41, 6] Predicted [5, 17, 21, 27, 31, 41, 6]
Expected: [12, 27, 30, 31, 34, 48, 17] Predicted [12, 27, 30, 31, 34, 48, 17]
Expected: [11, 15, 24, 25, 26, 33, 42] Predicted [11, 15, 24, 25, 26, 33, 42]
Expected: [6, 24, 30, 43, 44, 45, 11] Predicted [6, 24, 30, 43, 44, 45, 11]
Expected

Expected: [21, 25, 33, 36, 42, 48, 13] Predicted [21, 25, 33, 36, 42, 48, 13]
Expected: [1, 7, 19, 27, 38, 47, 3] Predicted [1, 7, 19, 27, 38, 47, 3]
Expected: [5, 21, 33, 34, 44, 45, 40] Predicted [5, 21, 33, 34, 44, 45, 40]
Expected: [9, 13, 22, 35, 38, 39, 4] Predicted [9, 13, 22, 35, 38, 39, 4]
Expected: [9, 11, 23, 29, 33, 38, 6] Predicted [9, 11, 23, 29, 33, 38, 6]
Expected: [9, 13, 32, 38, 48, 49, 41] Predicted [9, 13, 32, 38, 48, 49, 41]
Expected: [1, 11, 21, 27, 37, 48, 18] Predicted [1, 11, 21, 27, 37, 48, 18]
Expected: [2, 19, 24, 39, 41, 49, 21] Predicted [2, 19, 24, 39, 41, 49, 21]
Expected: [3, 8, 13, 17, 22, 29, 18] Predicted [3, 8, 13, 17, 22, 29, 18]
Expected: [1, 3, 14, 26, 36, 47, 42] Predicted [1, 3, 14, 26, 36, 47, 42]
Expected: [5, 13, 20, 25, 29, 39, 35] Predicted [5, 13, 20, 25, 29, 39, 35]
Expected: [1, 3, 20, 23, 33, 37, 14] Predicted [1, 3, 20, 23, 33, 37, 14]
Expected: [4, 9, 17, 26, 40, 46, 18] Predicted [4, 9, 17, 26, 40, 46, 18]
Expected: [4, 14, 19, 21, 

Expected: [1, 14, 28, 41, 45, 47, 29] Predicted [1, 14, 28, 41, 45, 47, 29]
Expected: [9, 27, 29, 38, 39, 40, 47] Predicted [9, 27, 29, 38, 39, 40, 47]
Expected: [1, 22, 34, 35, 36, 47, 7] Predicted [1, 22, 34, 35, 36, 47, 7]
Expected: [10, 16, 21, 30, 33, 48, 19] Predicted [10, 16, 21, 30, 33, 48, 19]
Expected: [12, 17, 24, 35, 44, 46, 10] Predicted [12, 17, 24, 35, 44, 46, 10]
Expected: [5, 9, 16, 29, 36, 47, 17] Predicted [5, 9, 16, 29, 36, 47, 17]
Expected: [2, 9, 29, 31, 42, 49, 5] Predicted [2, 9, 29, 31, 42, 49, 5]
Expected: [17, 34, 36, 41, 46, 48, 26] Predicted [17, 34, 36, 41, 46, 48, 26]
Expected: [3, 5, 7, 18, 40, 45, 15] Predicted [3, 5, 7, 18, 40, 45, 15]
Expected: [2, 6, 17, 33, 42, 45, 13] Predicted [2, 6, 17, 33, 42, 45, 13]
Expected: [4, 15, 16, 22, 24, 45, 2] Predicted [4, 15, 16, 22, 24, 45, 2]
Expected: [5, 6, 7, 18, 27, 35, 19] Predicted [5, 6, 7, 18, 27, 35, 19]
Expected: [4, 5, 17, 39, 40, 42, 3] Predicted [4, 5, 17, 39, 40, 42, 3]
Expected: [4, 5, 33, 36, 48, 4

Expected: [1, 14, 39, 40, 45, 49, 42] Predicted [1, 14, 39, 40, 45, 49, 42]
Expected: [21, 23, 26, 36, 45, 49, 2] Predicted [21, 23, 26, 36, 45, 49, 2]
Expected: [1, 11, 23, 37, 42, 43, 24] Predicted [1, 11, 23, 37, 42, 43, 24]
Expected: [6, 9, 12, 15, 43, 47, 49] Predicted [6, 9, 12, 15, 43, 47, 49]
Expected: [2, 5, 6, 12, 22, 40, 24] Predicted [2, 5, 6, 12, 22, 40, 24]
Expected: [12, 21, 24, 32, 36, 46, 13] Predicted [12, 21, 24, 32, 36, 46, 13]
Expected: [2, 22, 23, 40, 44, 45, 32] Predicted [2, 22, 23, 40, 44, 45, 32]
Expected: [16, 17, 21, 28, 38, 45, 10] Predicted [16, 17, 21, 28, 38, 45, 10]
Expected: [9, 22, 30, 41, 42, 48, 49] Predicted [9, 22, 30, 41, 42, 48, 49]
Expected: [13, 33, 35, 39, 40, 48, 27] Predicted [13, 33, 35, 39, 40, 48, 27]
Expected: [5, 7, 8, 23, 36, 41, 24] Predicted [5, 7, 8, 23, 36, 41, 24]
Expected: [6, 9, 13, 31, 40, 45, 47] Predicted [6, 9, 13, 31, 40, 45, 47]
Expected: [4, 8, 24, 27, 35, 45, 15] Predicted [4, 8, 24, 27, 35, 45, 15]
Expected: [11, 14, 1

...

Expected: [5, 7, 8, 18, 24, 29, 14] Predicted [5, 7, 8, 18, 24, 29, 14]
Expected: [5, 14, 31, 32, 36, 46, 11] Predicted [5, 14, 31, 32, 36, 46, 11]
Expected: [1, 9, 15, 31, 33, 41, 37] Predicted [1, 9, 15, 31, 33, 41, 37]
Expected: [7, 9, 27, 31, 36, 39, 32] Predicted [7, 9, 27, 31, 36, 39, 32]
Expected: [4, 8, 14, 23, 48, 49, 11] Predicted [4, 8, 14, 23, 48, 49, 11]
Expected: [23, 26, 27, 37, 42, 46, 15] Predicted [23, 26, 27, 37, 42, 46, 15]
Expected: [2, 5, 27, 32, 42, 46, 16] Predicted [2, 5, 27, 32, 42, 46, 16]
Expected: [10, 14, 26, 29, 30, 31, 18] Predicted [10, 14, 26, 29, 30, 31, 18]
Expected: [4, 12, 14, 24, 32, 38, 13] Predicted [4, 12, 14, 24, 32, 38, 13]
Expected: [25, 29, 32, 33, 37, 44, 47] Predicted [25, 29, 32, 33, 37, 44, 47]
Expected: [11, 17, 24, 27, 31, 38, 49] Predicted [11, 17, 24, 27, 31, 38, 49]
Expected: [15, 18, 24, 27, 39, 49, 6] Predicted [15, 18, 24, 27, 39, 49, 6]
Expected: [26, 28, 37, 43, 45, 47, 25] Predicted [26, 28, 37, 43, 45, 47, 25]
Expected: [11,

Expected: [9, 16, 28, 36, 42, 45, 2] Predicted [9, 16, 28, 36, 42, 45, 2]
Expected: [11, 12, 15, 21, 38, 42, 25] Predicted [11, 12, 15, 21, 38, 42, 25]
Expected: [18, 24, 28, 31, 39, 49, 44] Predicted [18, 24, 28, 31, 39, 49, 44]
Expected: [9, 11, 14, 22, 27, 49, 44] Predicted [9, 11, 14, 22, 27, 49, 44]
Expected: [7, 12, 16, 17, 20, 25, 31] Predicted [7, 12, 16, 17, 20, 25, 31]
Expected: [10, 13, 35, 38, 42, 44, 27] Predicted [10, 13, 35, 38, 42, 44, 27]
Expected: [10, 12, 24, 25, 26, 33, 23] Predicted [10, 12, 24, 25, 26, 33, 23]
Expected: [13, 16, 24, 26, 40, 44, 42] Predicted [13, 16, 24, 26, 40, 44, 42]
Expected: [4, 11, 12, 24, 35, 46, 3] Predicted [4, 11, 12, 24, 35, 46, 3]
Expected: [12, 16, 31, 34, 42, 48, 37] Predicted [12, 16, 31, 34, 42, 48, 37]
Expected: [3, 10, 26, 33, 42, 46, 44] Predicted [3, 10, 26, 33, 42, 46, 44]
Expected: [1, 5, 26, 28, 40, 49, 15] Predicted [1, 5, 26, 28, 40, 49, 15]
Expected: [8, 9, 18, 33, 41, 43, 15] Predicted [8, 9, 18, 33, 41, 43, 15]
Expected

Expected: [17, 24, 38, 44, 46, 47, 27] Predicted [17, 24, 38, 44, 46, 47, 27]
Expected: [9, 12, 13, 22, 30, 31, 28] Predicted [9, 12, 13, 22, 30, 31, 28]
Expected: [4, 6, 29, 34, 36, 45, 47] Predicted [4, 6, 29, 34, 36, 45, 47]
Expected: [4, 10, 21, 30, 35, 43, 7] Predicted [4, 10, 21, 30, 35, 43, 7]
Expected: [8, 21, 25, 32, 34, 45, 31] Predicted [8, 21, 25, 32, 34, 45, 31]
Expected: [3, 6, 13, 38, 43, 44, 31] Predicted [3, 6, 13, 38, 43, 44, 31]
Expected: [21, 25, 28, 33, 46, 48, 10] Predicted [21, 25, 28, 33, 46, 48, 10]
Expected: [14, 21, 25, 27, 44, 46, 3] Predicted [14, 21, 25, 27, 44, 46, 3]
Expected: [16, 24, 28, 35, 45, 46, 38] Predicted [16, 24, 28, 35, 45, 46, 38]
Expected: [5, 8, 30, 34, 45, 48, 3] Predicted [5, 8, 30, 34, 45, 48, 3]
Expected: [3, 5, 14, 15, 39, 41, 21] Predicted [3, 5, 14, 15, 39, 41, 21]
Expected: [2, 10, 16, 35, 46, 47, 49] Predicted [2, 10, 16, 35, 46, 47, 49]
Expected: [1, 3, 25, 27, 28, 45, 12] Predicted [1, 3, 25, 27, 28, 45, 12]
Expected: [11, 20, 2

Expected: [2, 5, 23, 27, 36, 43, 20] Predicted [2, 5, 23, 27, 36, 43, 20]
Expected: [13, 24, 28, 29, 31, 44, 10] Predicted [13, 24, 28, 29, 31, 44, 10]
Expected: [8, 15, 25, 30, 34, 40, 42] Predicted [8, 15, 25, 30, 34, 40, 42]
Expected: [13, 14, 18, 23, 29, 35, 45] Predicted [13, 14, 18, 23, 29, 35, 45]
Expected: [7, 14, 25, 28, 33, 40, 26] Predicted [7, 14, 25, 28, 33, 40, 26]
Expected: [6, 11, 22, 25, 32, 33, 24] Predicted [6, 11, 22, 25, 32, 33, 24]
Expected: [3, 12, 13, 19, 24, 45, 40] Predicted [3, 12, 13, 19, 24, 45, 40]
Expected: [7, 21, 26, 30, 38, 47, 15] Predicted [7, 21, 26, 30, 38, 47, 15]
Expected: [14, 16, 21, 28, 43, 46, 26] Predicted [14, 16, 21, 28, 43, 46, 26]
Expected: [5, 8, 23, 34, 41, 44, 7] Predicted [5, 8, 23, 34, 41, 44, 7]
Expected: [5, 8, 12, 17, 20, 37, 45] Predicted [5, 8, 12, 17, 20, 37, 45]
Expected: [1, 7, 13, 29, 32, 44, 20] Predicted [1, 7, 13, 29, 32, 44, 20]
Expected: [13, 15, 16, 32, 34, 46, 10] Predicted [13, 15, 16, 32, 34, 46, 10]
Expected: [4, 

Expected: [19, 24, 26, 30, 36, 41, 43] Predicted [19, 24, 26, 30, 36, 41, 43]
Expected: [8, 27, 30, 35, 36, 43, 31] Predicted [8, 27, 30, 35, 36, 43, 31]
Expected: [3, 4, 30, 37, 39, 42, 32] Predicted [3, 4, 30, 37, 39, 42, 32]
Expected: [5, 13, 16, 28, 40, 41, 2] Predicted [5, 13, 16, 28, 40, 41, 2]
Expected: [6, 8, 9, 23, 36, 40, 19] Predicted [6, 8, 9, 23, 36, 40, 19]
Expected: [4, 10, 32, 35, 38, 47, 49] Predicted [4, 10, 32, 35, 38, 47, 49]
Expected: [7, 14, 19, 27, 28, 38, 46] Predicted [7, 14, 19, 27, 28, 38, 46]
Expected: [7, 9, 14, 18, 20, 44, 29] Predicted [7, 9, 14, 18, 20, 44, 29]
Expected: [1, 25, 34, 37, 39, 47, 18] Predicted [1, 25, 34, 37, 39, 47, 18]
Expected: [22, 24, 29, 35, 38, 48, 30] Predicted [22, 24, 29, 35, 38, 48, 30]
Expected: [4, 15, 29, 34, 38, 48, 30] Predicted [4, 15, 29, 34, 38, 48, 30]
Expected: [7, 10, 20, 25, 34, 35, 8] Predicted [7, 10, 20, 25, 34, 35, 8]
Expected: [1, 3, 6, 9, 24, 28, 36] Predicted [1, 3, 6, 9, 24, 28, 36]
Expected: [3, 9, 33, 38, 4

In [206]:
# evaluate LSTM
y_predict = model.predict(X, batch_size=batch_size, verbose=0)
# decode all pairs
for i in range(len(X)):
    print('Expected:', one_hot_decode(y[i]), 'Predicted', one_hot_decode(y_predict[i]))

NameError: name 'X' is not defined

In [110]:
#print(one_hot_decode(y_predict[len(y_predict)-1]))
print(one_hot_decode(testX[len(testX)-1]))
print(one_hot_decode(y_predict[len(testX)-1]))

[7, 13, 22, 27, 33, 36, 45]


NameError: name 'y_predict' is not defined

In [207]:
y_predict = model.predict(one_hot_encode([['6', '10', '32', '36', '41', '48', '47']]), batch_size=batch_size, verbose=0)

Row: ['6', '10', '32', '36', '41', '48', '47']
Encoding Row:  [[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 

RuntimeError: FindSequence: Requested sequence (id 1) not found.

[CALL STACK]
    > CNTK::TrainingParameterSchedule<unsigned __int64>::  GetMinibatchSize
    - CNTK::TrainingParameterSchedule<unsigned __int64>::  GetMinibatchSize (x3)
    - CNTK::Internal::  UseSparseGradientAggregationInDataParallelSGD
    - CNTK::TrainingParameterSchedule<unsigned __int64>::  GetMinibatchSize
    - CNTK::Internal::  UseSparseGradientAggregationInDataParallelSGD
    - CNTK::Function::  Forward
    - PyInit__cntk_py (x2)
    - PyCFunction_Call
    - PyEval_GetFuncDesc
    - PyEval_EvalFrameEx (x2)
    - PyEval_GetFuncDesc (x2)



In [21]:
# split into train and test sets -> 7:3
import numpy as np

train_size = int(len(encoded) * 0.70)
test_size = len(encoded) - train_size
train, test = encoded[0:train_size,:], encoded[train_size:len(encoded),:]
print("length of dataset(total lottery draws): ", len(encoded), "\nlength of train dataset: ", len(train), "\nlength of test dataset: ", len(test))
#print(train[len(train)-1])

# create a new dataset where X is the Closing Price of HSBC at a given time (t) and Y is the Closing Price at the next time (t + 1)
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    #for i in range(len(dataset)-look_back-1):
    for i in range(len(dataset)-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

print(trainX[len(trainX)-1])
#print(testY[len(testY)-1])

length of dataset(total lottery draws):  2379 
length of train dataset:  1665 
length of test dataset:  714
[[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
